In [1]:
from data_prep import *
from model import *

### data preparation 

First things first. Let's start from data preparation. It's quite minimal here and the main question is how do we construct our batches. 

We read the text as it is without preprocessing or normalizing and extract all chars from it. So we have a lot of chars and we organize them using a dictionary where each char has an index. We also encode text as a list of numbers where each number is an index of a char in the mentioned dictionary.

In [6]:
with open('data/anna.txt', 'r') as f:
    text = f.read()

In [8]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [21]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
print(list(char2int.items())[:5]), print(list(char2int.items())[-5:])

[('b', 0), ('p', 1), ('1', 2), ('O', 3), ('F', 4)]
[('s', 78), ('S', 79), ('t', 80), ('q', 81), ('M', 82)]


(None, None)

In [13]:
encoded = np.array([char2int[ch] for ch in text])

In [14]:
encoded[:10]

array([22, 52, 26,  1, 80, 43, 16, 28,  2, 19])

In [15]:
[int2char[i] for i in encoded[:10]]

['C', 'h', 'a', 'p', 't', 'e', 'r', ' ', '1', '\n']

Next step - we do one-hot encoding. Function mechanics is a bit involved but the result is quite clear.

In [19]:
arr = encoded[:10].reshape(2, 5)
arr, arr.shape

(array([[22, 52, 26,  1, 80],
        [43, 16, 28,  2, 19]]), (2, 5))

In [17]:
one_hot = one_hot_encode(arr=arr, n_labels=len(chars))

In [18]:
one_hot.shape

(2, 5, 83)

In [24]:
one_hot[0, 0, :]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [25]:
one_hot[0, 0, 22]

1.0

Now it's the most interesting part - batch construction. We have 2 new parameters: `batch_size=128` and `seq_length=100`. So we're going to reshape `encoded` into `128` rows and then use a sliding window of size `100` to iterate over it. 

First question - why don't we construct a batch using sequntial values of `encoded`? Well probably because it doesn't really matter which way to construct. This is char RNN after all. We need to predict next char based on some previos ones and `seq_length` is quite high to make all rows of a batch completely independent. But this is just a hypothesis. In fact we have stateful and stateless `RNNs` - see *Hands-On Machine Leaning, ch. 16*. And in stateful `RNNs` the question how to build a batch does matter.

Let's use much less values of parameters for illustration purposes.

In [26]:
batch_size, seq_length = 3, 5

In [28]:
arr = encoded
n_batches = int(arr.size / (batch_size * seq_length))
arr = arr[:(n_batches * batch_size * seq_length)]
arr = arr.reshape((batch_size, -1))

In [29]:
arr.shape

(3, 661740)

In [31]:
arr[:, :10]

array([[22, 52, 26,  1, 80, 43, 16, 28,  2, 19],
       [17,  0, 66, 43, 60, 80, 45, 19, 19, 39],
       [42, 43, 28, 27, 38, 43, 16, 19, 26, 65]])

How should our batches look like? It's important that a sliding window goes in `seq_length` steps.

In [34]:
arr[:, :5]

array([[22, 52, 26,  1, 80],
       [17,  0, 66, 43, 60],
       [42, 43, 28, 27, 38]])

In [37]:
arr[:, 5:10]

array([[43, 16, 28,  2, 19],
       [80, 45, 19, 19, 39],
       [43, 16, 19, 26, 65]])

In [45]:
batch_generator = get_batches(arr=encoded, batch_size=3, seq_length=5)

In [46]:
x, y = next(batch_generator)
x

array([[22, 52, 26,  1, 80],
       [17,  0, 66, 43, 60],
       [42, 43, 28, 27, 38]])

In [47]:
x, _ = next(batch_generator)
x

array([[43, 16, 28,  2, 19],
       [80, 45, 19, 19, 39],
       [43, 16, 19, 26, 65]])

Last question - how do we compute `y`? It should be shifted by one position - we predict next character in a sequence after all. So if first row is `[22, 52, 26,  1, 80, 43, 16, 28,  2, 19, ...]` then the first row of `x` is `[22, 52, 26,  1, 80]`  and the first row of `y` is `[52, 26, 1, 80, 43]`.

In [44]:
arr[:, 1:6]

array([[52, 26,  1, 80, 43],
       [ 0, 66, 43, 60, 80],
       [43, 28, 27, 38, 43]])

In [48]:
y

array([[52, 26,  1, 80, 43],
       [ 0, 66, 43, 60, 80],
       [43, 28, 27, 38, 43]])

### lstm model

Next question - let's decompose our model bit-by-bit and simulate forward pass. There're a lot of questions out there - where do we use `seq_length` in `LSTM`, what is output of `LSTM`, do we use `softmax` or not, can we handle sequences of variable length and so on.

First question - what exactly do we need to construct `LSTM`? Let's start with `LSTMCell`. All the gates in `LSTMCell` are basically linear layers with some activation. So we have matrices of weights that depend on 2 parameters: `input_size` and `hidden_size`. And these are exactly parameters that we need to instantiate `LSTMCell`.

Now to instantiate `LSTM` we need only one additional parameter - `n_layers`. In our case `n_layers=2` and that means that we have 2 layers of LSTM stacked together. So here's the question - how do `LSTM` handle unrolling? How does it know `seq_length`?   

In [145]:
batch_size, seq_length, n_chars = 3, 5, 83
batch_gen = get_tensor_batches(encoded, batch_size, seq_length, n_chars)
x, y = next(batch_gen)
x.shape, y.shape

(torch.Size([3, 5, 83]), torch.Size([3, 5]))

In [146]:
y

tensor([[52, 26,  1, 80, 43],
        [ 0, 66, 43, 60, 80],
        [43, 28, 27, 38, 43]])

Let's create a simple `LSTM` with just `1` layer. Let's use `bias=False` to simplify computations.

What does `batch_first=True` mean? From documentation: *If `True`, then the input and output tensors are provided as `(batch, seq, feature)`.* And this is indeed the case: `x.shape[0]` is in fact the `batch_size`.

In [186]:
torch.manual_seed(0)
lstm_cell = nn.LSTMCell(input_size=len(chars),
                       hidden_size=2,
                       bias=False)
torch.manual_seed(0)
lstm = nn.LSTM(input_size=len(chars), 
               hidden_size=2, 
               num_layers=1,
               dropout=0, 
               batch_first=True,
               bias=False)

In [111]:
# shapes should be (4 * hidden_size, input_size)
# and (4 * hidden_size, hidden_size)
print(lstm_cell.weight_ih.shape, lstm_cell.weight_hh.shape)
print(lstm.weight_ih_l0.shape, lstm.weight_hh_l0.shape)

torch.Size([8, 83]) torch.Size([8, 2])
torch.Size([8, 83]) torch.Size([8, 2])


Next question - how should we initialize our hidden state? It turns out that most popular solution is just set it to `0` (again, see an alternative in *Hands-On Machine Leaning, ch. 16*). And it seems this option is now available by default, from documentation: *If `(h_0, c_0)` is not provided, both `h_0` and `c_0` default to zero.* This was not alwais the case: see [here](https://github.com/pytorch/pytorch/issues/434).

That's easy to check directly: below we do computations with default params and with `(h0, c0)` set to `0` - result is the same.

#### computations of `lstm_cell`

So let's try to reproduce computations of the `LSTMCell`. 

In [149]:
x.shape

torch.Size([3, 5, 83])

In [148]:
x0 = x[:, 0, :]
x0.shape

torch.Size([3, 83])

In [150]:
# let's check that we don't have bias
lstm_cell.bias_ih, lstm_cell.bias_hh

(None, None)

In [197]:
Wii, Wif, Wig, Wio = lstm_cell.weight_ih[0:2, :], lstm_cell.weight_ih[2:4, :], \
                     lstm_cell.weight_ih[4:6, :], lstm_cell.weight_ih[6:8, :]
Wii.shape, Wif.shape, Wig.shape, Wio.shape

(torch.Size([2, 83]),
 torch.Size([2, 83]),
 torch.Size([2, 83]),
 torch.Size([2, 83]))

In [216]:
with torch.no_grad():
    i = torch.sigmoid(torch.mm(x0, Wii.t()))
    g = torch.tanh(torch.mm(x0, Wig.t()))
    o = torch.sigmoid(torch.mm(x0, Wio.t()))
    print(i.shape, g.shape, o.shape)

    c_man = i * g
    h_man = o * torch.tanh(c_man)
    print(h_man.shape, c_man.shape)

torch.Size([3, 2]) torch.Size([3, 2]) torch.Size([3, 2])
torch.Size([3, 2]) torch.Size([3, 2])


In [217]:
h_man

tensor([[ 0.0320, -0.1586],
        [ 0.0677,  0.2253],
        [-0.1245,  0.0426]])

In [218]:
c_man

tensor([[ 0.0579, -0.2446],
        [ 0.1210,  0.3548],
        [-0.3145,  0.1283]])

In [155]:
h0, c0 = torch.zeros(3, 2), torch.zeros(3, 2)
h0, c0

(tensor([[0., 0.],
         [0., 0.],
         [0., 0.]]), tensor([[0., 0.],
         [0., 0.],
         [0., 0.]]))

In [209]:
with torch.no_grad():
    h_comp, c_comp = lstm_cell(x0, (h0, c0))

In [210]:
h_comp

tensor([[ 0.0320, -0.1586],
        [ 0.0677,  0.2253],
        [-0.1245,  0.0426]])

In [211]:
c_comp

tensor([[ 0.0579, -0.2446],
        [ 0.1210,  0.3548],
        [-0.3145,  0.1283]])

#### computations of `lstm`

And now is the **hard** part - computations of the `LSTM`. Probably we may use `LSTMCell` and a loop rather than do all computations manually. For convenience let's define all inputs and models one more time.